## Fake News Headline Detection using LSTM 

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,GRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
fake_news=pd.read_csv("C:/Users/acer/Downloads/fake-and-real-news-dataset/Fake.csv")
real_news=pd.read_csv("C:/Users/acer/Downloads/fake-and-real-news-dataset/True.csv")

In [3]:
fake_news

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [4]:
fake_news["Label"]= "Fake"

In [5]:
fake_news

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


In [6]:
real_news["Label"]= "Real"

In [7]:
real_news

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",Real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",Real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",Real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",Real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",Real
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",Real
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",Real
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",Real
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",Real


In [8]:
real_news=real_news.drop(["text","subject","date"],axis=1)

In [9]:
fake_news=fake_news.drop(["text","subject","date"],axis=1)

In [10]:
real_news

,title,Label
0,"As U.S. budget fight looms, Republicans flip t...",Real
1,U.S. military to accept transgender recruits o...,Real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,Real
3,FBI Russia probe helped by Australian diplomat...,Real
4,Trump wants Postal Service to charge 'much mor...,Real
...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,Real
21413,LexisNexis withdrew two products from Chinese ...,Real
21414,Minsk cultural hub becomes haven from authorities,Real
21415,Vatican upbeat on possibility of Pope Francis ...,Real


In [11]:
fake_news

,title,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Fake
1,Drunk Bragging Trump Staffer Started Russian ...,Fake
2,Sheriff David Clarke Becomes An Internet Joke...,Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Fake
...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Fake
23479,How to Blow $700 Million: Al Jazeera America F...,Fake


In [12]:
news=pd.concat([real_news,fake_news],axis=0)

In [13]:
news = news.sample(frac=1).reset_index(drop=True)

In [14]:
news.head(20)

,title,Label
0,"BILLIONAIRE, Leftist Author Of Harry Potter Bo...",Fake
1,"Obama, trying to protect legacy, unlikely to a...",Real
2,RUBIO Sides With Democrats On Giving A Whoppin...,Fake
3,Trump national security adviser Flynn resigns ...,Real
4,Former Ambassador Accuses Trump Of Violating ...,Fake
5,Secret Report: Top Russians Celebrated Trump Win,Fake
6,EU Commission 'horrified' by killing of Maltes...,Real
7,South Africa's Zuma asks court to set aside re...,Real
8,Trump Encourages Audience To Call Ted Cruz A ...,Fake
9,Pelosi re-elected to House Democratic leadersh...,Real


In [15]:
X=news["title"]
Y=news["Label"]
le=preprocessing.LabelEncoder()
Y=le.fit_transform(Y)
Y=Y.reshape(-1,1)

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.25,random_state = 0)

In [17]:
max_words = 500
max_len = 150
token = Tokenizer(num_words=max_words)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
x_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [18]:
x_matrix

array([[  0,   0,   0, ...,  10,  12, 184],
       [  0,   0,   0, ...,   2,  21, 154],
       [  0,   0,   0, ...,  11, 294,   4],
       ...,
       [  0,   0,   0, ...,   1, 289, 386],
       [  0,   0,   0, ..., 226,  22, 289],
       [  0,   0,   0, ..., 322,   8,   4]])

In [19]:
def FND():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = GRU(70, return_sequences=True)(layer)
    layer = LSTM(70)(layer)
    layer = Activation('relu')(layer)
    layer = Dense(110)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [20]:
model = FND()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [21]:
model.fit(x_matrix,y_train,epochs=10, validation_split=0.2)

Train on 26938 samples, validate on 6735 samples
Epoch 1/10
26938/26938 [==============================] - 156s 6ms/sample - loss: 0.2345 - accuracy: 0.9091 - val_loss: 0.1604 - val_accuracy: 0.9381
Epoch 2/10
26938/26938 [==============================] - 148s 5ms/sample - loss: 0.1718 - accuracy: 0.9344 - val_loss: 0.1502 - val_accuracy: 0.9422
Epoch 3/10
26938/26938 [==============================] - 151s 6ms/sample - loss: 0.1619 - accuracy: 0.9371 - val_loss: 0.1448 - val_accuracy: 0.9454
Epoch 4/10
26938/26938 [==============================] - 155s 6ms/sample - loss: 0.1554 - accuracy: 0.9396 - val_loss: 0.1433 - val_accuracy: 0.9465
Epoch 5/10
26938/26938 [==============================] - 157s 6ms/sample - loss: 0.1522 - accuracy: 0.9406 - val_loss: 0.1502 - val_accuracy: 0.9454
Epoch 6/10
26938/26938 [==============================] - 149s 6ms/sample - loss: 0.1475 - accuracy: 0.9436 - val_loss: 0.1616 - val_accuracy: 0.9384
Epoch 7/10
26938/26938 [===========================

In [22]:
max_words = 500
max_len = 150
token = Tokenizer(num_words=max_words)
token.fit_on_texts(X_test)
sequences = token.texts_to_sequences(X_test)
xt_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [23]:
acc=model.evaluate(xt_matrix,y_test,verbose=0)

In [24]:
print("Accuracy of model is:",acc[1]*100)

Accuracy of model is: 72.72160649299622
